In [1]:
import requests
import pandas as pd
import geopandas as gpd
import math
from datetime import date
import datetime
import time


### Querying data from https://docs.openaq.org/docs/getting-started

In [24]:
def get_sensor_locations_in_canada( country_code = "CA", parameter = "pm25", limit = 1000):
  # ISO 3166-1 alpha-2 country code for Montenegro

    if(limit != 1000):
        print("Warning, it's possible that this function only works with max limit 1000, and behaves weirdly otherwise")
        
    url = f"https://api.openaq.org/v2/locations?country={country_code}&parameter={parameter}&limit=1000"#&offset=1000"
    #url = f"https://api.openaq.org/v2/locations?country={country_code}&parameters_id=2&limit=1000"#&offset=1000"

    headers = {"accept": "application/json", "X-API-Key": "659706290b9f942d7b7e22e0e1667afc43164846ee91949f19f5a1312d2543e4"}

    response = requests.get(url, headers=headers)
    if(response.status_code != 200):
        print(response.text)

    resj = response.json()
    
    foo = pd.DataFrame(resj['results'])
    tot_pages = resj['meta']['found']
    print(tot_pages)
    div = tot_pages/limit
    pages = math.ceil(div)
    
    #list_of_locations = pd.DataFrame(test['results'])
    #tmp_frame = pd.DataFrame(test['results'])
    
    #list_of_locations = []
    
    df_raw = pd.DataFrame()
    
    for i in range(1, (pages+2)):
        print(i)
        url = f"https://api.openaq.org/v2/locations?country={country_code}&parameter={parameter}&limit={limit}&page={i}"
        response2 = requests.get(url, headers=headers)
        test = response2.json()
        tmp_frame = pd.DataFrame(test['results'])
        #print(tmp_frame)
        df_raw = pd.concat([ df_raw,  tmp_frame], ignore_index = True, sort = False)
        
#     num_pages = 1
#     while(len(tmp_frame) > 0): #(test['meta']['found'] + (limit*2)) > (test['meta']['limit'] * num_pages)
        
#         #print(test['meta']['found'],  (test['meta']['limit'] * num_pages))
#         print(num_pages)
#         url = f"https://api.openaq.org/v2/locations?country={country_code}&parameter={parameter}&limit={limit}&page={num_pages}"
#         response = requests.get(url, headers=headers)
#         test = response.json()
#         tmp_frame = pd.DataFrame(test['results'])      
#         list_of_locations = pd.concat([list_of_locations,  tmp_frame])
#         num_pages =  num_pages + 1
        
        
    return(df_raw)

def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].items()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].items())).fillna(fillna)], axis=1)
        del ret[column]
    return ret
    

,Unnamed: 0,none_found
0,0,383253
1,1,223021
2,2,399197
3,3,635630
4,4,1820911
5,5,635693
6,6,496097
7,7,382795
8,8,635657
9,9,1996960


In [84]:
class CustomException(Exception):
    pass


def get_measument_per_location(locations_id, limit = 1000):
    time.sleep(3)
    if(limit != 1000):
        print("Warning, it's possible that this function only works with max limit 1000, and behaves weirdly otherwise")
        
    url = f"https://api.openaq.org/v2/averages?parameters_id=2&date_to=2023-09-30T00%3A00%3A00&date_from=2023-05-01T00%3A00%3A00&country=CA&locations_id={locations_id}&limit=1&page=1"

    headers = {"accept": "application/json", "X-API-Key": "659706290b9f942d7b7e22e0e1667afc43164846ee91949f19f5a1312d2543e4"}

    response = requests.get(url, headers=headers)
    
    if(response.status_code != 200):
        print(response.text)
        if(response.status_code == 429):
            return(None)
            #time.sleep(300)
        

    test = response.json()
    
    
    
    
    #list_of_locations = pd.DataFrame(test['results'])
    #tmp_frame = pd.DataFrame(test['results'])
    
    #list_of_locations = []
    
    df_raw = pd.DataFrame()
    if(test['meta']['found'] == 0):
        print(test)
        print("no entries found for id", locations_id)
        raise CustomException(" ")
    page = 1
    while ((test['meta']['found'] != 0)):
        time.sleep(1)
        #print(page)
        url = f"https://api.openaq.org/v2/averages?parameters_id=2&date_to=2023-09-30T00%3A00%3A00&date_from=2023-05-01T00%3A00%3A00&country=CA&locations_id={locations_id}&limit=1000&page={page}"
        response2 = requests.get(url, headers=headers)
        if(response2.status_code != 200):
            print(response2.text)
            if(response2.status_code == 429):
                return(None)
               # time.sleep(300)
        test = response2.json()
        
        #print(test['meta']['found'])
        tmp_frame = pd.DataFrame(test['results'])
        #print(tmp_frame)
        df_raw = pd.concat([ df_raw,  tmp_frame], ignore_index = True, sort = False)
        page = page + 1
        
        
    return(df_raw)

def get_measument_per_location_special(locations_id, limit = 1000):
    '''
    Get measurment per location. Different than previous function because it will query over a larger timeframe, and then subset the result. 
    '''
    #time.sleep(1)
    if(limit != 1000):
        print("Warning, it's possible that this function only works with max limit 1000, and behaves weirdly otherwise")
        
    url = f"https://api.openaq.org/v2/averages?parameters_id=2&date_to=2024-02-07T00%3A00%3A00&date_from=2023-05-01T00%3A00%3A00&country=CA&locations_id={locations_id}&limit=1&page=1"

    headers = {"accept": "application/json", "X-API-Key": "659706290b9f942d7b7e22e0e1667afc43164846ee91949f19f5a1312d2543e4"}

    response = requests.get(url, headers=headers)
    
    if(response.status_code != 200):
        print(response.text)
        if(response.status_code == 429):
            return(None)
            #time.sleep(300)
        

    test = response.json()
    
    
    
    
    #list_of_locations = pd.DataFrame(test['results'])
    #tmp_frame = pd.DataFrame(test['results'])
    
    #list_of_locations = []
    
    df_raw = pd.DataFrame()
    if(test['meta']['found'] == 0):
        print(test)
        print("no entries found for id", locations_id)
        raise CustomException(" ")
    page = 1
    while ((test['meta']['found'] != 0)):
        #time.sleep(1)
        #print(page)
        url = f"https://api.openaq.org/v2/averages?parameters_id=2&date_to=2024-02-07T00%3A00%3A00&date_from=2023-05-01T00%3A00%3A00&country=CA&locations_id={locations_id}&limit=1000&page={page}"
        response2 = requests.get(url, headers=headers)
        if(response2.status_code != 200):
            print(response2.text)
            if(response2.status_code == 429):
                return(None)
               # time.sleep(300)
        test = response2.json()
        
        #print(test['meta']['found'])
        tmp_frame = pd.DataFrame(test['results'])
        #print(tmp_frame)
        df_raw = pd.concat([ df_raw,  tmp_frame], ignore_index = True, sort = False)
        page = page + 1
        
    df_raw[(df_raw.hour <= "2023-09-30 23:59:59") & (df_raw.hour >= "2023-05-01 00:00:00")]
    return(df_raw)

In [26]:
locs = get_sensor_locations_in_canada()

locs.to_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/" + "Locations" + str(date.today().strftime("%Y%m%d")) +".csv")
print(len(locs.id.unique()))

2276
1
2
3
4
2265


In [42]:
### Find number of IDS in the time range. (Not actually sure how strict the querying is, it might not drop IDS based on if they only partially overlap. Need to check). 

#2023-09-30T00%3A00%3A00&date_from=2023-05-01T00%3A00%3A00

#start_time = datetime.datetime.strptime("2023-05-01 00:00:00", "%Y-%m-%d %H:%M:%S")
#end_time = datetime.datetime.strptime("2023-09-30 00:00:00", "%Y-%m-%d %H:%M:%S")
start_time = "2023-05-01 00:00:00+00:00"
end_time = "2023-09-30 00:00:00+00:00"




len(locs[(locs.firstUpdated.astype('datetime64[ns, UTC]') <= start_time) & (locs.lastUpdated.astype('datetime64[ns, UTC]') >= end_time)].id.unique())

locs_time = locs[(locs.firstUpdated.astype('datetime64[ns, UTC]') <= start_time) & (locs.lastUpdated.astype('datetime64[ns, UTC]') >= end_time)]

In [46]:
ids = locs.id.unique()

all_measurments = pd.DataFrame()
no_entries_id = list()


for n,i in enumerate(ids, start = 0):
    if(n == 0):
        print("n = 0")
    elif((n%150) == 0):
        time.sleep(3)
    try:
        foo = get_measument_per_location(i)
    except CustomException as e:
        no_entries_id.append(i)
        
    except Exception as e:
        print("Error at ID: ",i)
        print(e)
        continue
    ## Extract the period between 
    all_measurments = pd.concat([all_measurments, foo])
    #print(fires)
        #fr_pd = pd.DataFrame(fires, columns=["lat", "lon", "farea", "data_source"])
    all_measurments.to_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/" + str(date.today().strftime("%Y%m%d"))+  ".csv")
    tmp_df = pd.DataFrame(no_entries_id, columns= ["none_found"])
    tmp_df.to_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/" + "missing_ids"+ str(date.today().strftime("%Y%m%d"))+  ".csv")

In [30]:
### Read in missing IDS from previous runs. These need to be handled differently becuase of the bug in the API

missing_ids = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/missing_ids20240201.csv")

missing_ids

In [89]:
## Check file to see if it processed all the IDS. If not, subset to missing IDs and try again. 

sensors = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/20240125.csv")
sensors2 = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/20240126.csv")
sensor3 = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/20240129.csv")
sensor4 = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/20240130.csv")
sensor5 = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/20240201.csv")
sensor6 = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/20240207.csv")

In [90]:
### Check just on the IDS where there is temporal overlap 

sub_missing_ids = locs_time.id[~locs_time.id.isin([*sensors.id.unique(), *sensors2.id.unique(), *sensor3.id.unique(),  *sensor5.id.unique(), *sensor6.id.unique()])]

In [91]:
print(sub_missing_ids[~sub_missing_ids.isin(missing_ids.none_found)])
print(~missing_ids.none_found[missing_ids.none_found.astype("str").isin(sub_missing_ids)])

new_missing_ids = [*missing_ids.none_found, *sub_missing_ids[~sub_missing_ids.isin(missing_ids.none_found)]]
#new_missing_ids

Series([], Name: id, dtype: int64)
Series([], Name: none_found, dtype: int64)


In [30]:


#tmp = get_measument_per_location("481")

In [43]:
#tmp[(tmp.hour <= "2023-09-30 23:59:59") & (tmp.hour >= "2023-05-01 00:00:00")]

,id,name,hour,day,month,year,hod,dow,average,measurement_count,parameter,parameterId,displayName,unit,first_datetime,last_datetime
0,481,Parc de la Gatineau,2023-05-01T05:00:00Z,2023-05-01,2023-05-01,2023-01-01,5,1,0.7,1,pm25,2,PM2.5,µg/m³,2023-05-01T05:00:00Z,2023-05-01T05:00:00Z
1,481,Parc de la Gatineau,2023-05-01T06:00:00Z,2023-05-01,2023-05-01,2023-01-01,6,1,0.7,1,pm25,2,PM2.5,µg/m³,2023-05-01T06:00:00Z,2023-05-01T06:00:00Z
2,481,Parc de la Gatineau,2023-05-01T07:00:00Z,2023-05-01,2023-05-01,2023-01-01,7,1,0.6,1,pm25,2,PM2.5,µg/m³,2023-05-01T07:00:00Z,2023-05-01T07:00:00Z
3,481,Parc de la Gatineau,2023-05-01T08:00:00Z,2023-05-01,2023-05-01,2023-01-01,8,1,0.4,1,pm25,2,PM2.5,µg/m³,2023-05-01T08:00:00Z,2023-05-01T08:00:00Z
4,481,Parc de la Gatineau,2023-05-01T09:00:00Z,2023-05-01,2023-05-01,2023-01-01,9,1,0.5,1,pm25,2,PM2.5,µg/m³,2023-05-01T09:00:00Z,2023-05-01T09:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3454,481,Parc de la Gatineau,2023-09-29T19:00:00Z,2023-09-29,2023-09-01,2023-01-01,19,5,14.0,1,pm25,2,PM2.5,µg/m³,2023-09-29T19:00:00Z,2023-09-29T19:00:00Z
3455,481,Parc de la Gatineau,2023-09-29T20:00:00Z,2023-09-29,2023-09-01,2023-01-01,20,5,15.0,1,pm25,2,PM2.5,µg/m³,2023-09-29T20:00:00Z,2023-09-29T20:00:00Z
3456,481,Parc de la Gatineau,2023-09-29T21:00:00Z,2023-09-29,2023-09-01,2023-01-01,21,5,15.0,1,pm25,2,PM2.5,µg/m³,2023-09-29T21:00:00Z,2023-09-29T21:00:00Z
3457,481,Parc de la Gatineau,2023-09-29T22:00:00Z,2023-09-29,2023-09-01,2023-01-01,22,5,15.0,1,pm25,2,PM2.5,µg/m³,2023-09-29T22:00:00Z,2023-09-29T22:00:00Z


In [86]:
all_measurments = pd.DataFrame()
no_entries_id = list()


for n,i in enumerate(new_missing_ids, start = 0):
    if(n == 0):
        print("n = 0")
    elif((n%150) == 0):
        time.sleep(3)
    try:
        foo = get_measument_per_location_special(i)
    except CustomException as e:
        no_entries_id.append(i)
        
    except Exception as e:
        print("Error at ID: ",i)
        print(e)
        continue
    ## Extract the period between 
    all_measurments = pd.concat([all_measurments, foo])
    #print(fires)
        #fr_pd = pd.DataFrame(fires, columns=["lat", "lon", "farea", "data_source"])
    all_measurments.to_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/" + str(date.today().strftime("%Y%m%d"))+  ".csv")
    tmp_df = pd.DataFrame(no_entries_id, columns= ["none_found"])
    tmp_df.to_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/" + "missing_ids"+ str(date.today().strftime("%Y%m%d"))+  ".csv")
    
    

n = 0
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 1380
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 1380


In [85]:
# #locations_id = 635657
# locations_id = 635599
# url = f"https://api.openaq.org/v2/averages?parameters_id=2&date_to=2024-02-07T00%3A00%3A00&date_from=2023-05-01T00%3A00%3A00&country=CA&locations_id={locations_id}&limit=1000&page=1"

# headers = {"accept": "application/json", "X-API-Key": "659706290b9f942d7b7e22e0e1667afc43164846ee91949f19f5a1312d2543e4"}

# response = requests.get(url, headers=headers)

# test = response.json()
# print(test)

In [5]:
# locations_id = 635657

# url = f"https://api.openaq.org/v2/averages?parameters_id=2&date_to=2023-09-30T00%3A00%3A00&date_from=2023-05-01T00%3A00%3A00&locations_id={locations_id}&limit=1000"

# headers = {"accept": "application/json"}

# response = requests.get(url, headers=headers)

# test = response.json()
# print(test)

In [19]:
# ids = locs.id.unique()

# all_measurments = pd.DataFrame()


# for n,i in enumerate(ids, start = 0):
#     try:
#         foo = get_measument_per_location(i)
#     except Exception as e:
#         print("Error at ID: ",i)
#         print(e)
#         continue
#     ## Extract the period between 
#     all_measurments = pd.concat([all_measurments, foo])
#     #print(fires)
#         #fr_pd = pd.DataFrame(fires, columns=["lat", "lon", "farea", "data_source"])
#     all_measurments.to_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/" + str(date.today().strftime("%Y%m%d"))+  ".csv")

In [164]:
# print(len(sensors.id.unique())) ## Doesn't have all the IDS
# print(len(sensors2.id.unique()))
# print(len(sensor3.id.unique()))
# print(len(sensor4.id.unique()))
# print(len(sensors.id.unique()) + len(sensors2.id.unique()) + len(sensor3.id.unique()))

In [169]:
# print(sensor4.id.unique())
# print(sensor3.id.unique())

[487 287 394 429 468 476]
[ 487  287  394  429  468  476  477  481  748  749 8132  482]


In [171]:
# ids = locs.id[~locs.id.isin([*sensors.id.unique(), *sensors2.id.unique(), *sensor3.id.unique(), *sensor4.id.unique() ])]

# print(len(ids))

662


In [87]:
locs[locs.id == 1380]

,id,city,name,entity,country,sources,isMobile,isAnalysis,parameters,sensorType,coordinates,lastUpdated,firstUpdated,measurements,bounds,manufacturers
1045,1380,MANITOBA,Winnipeg_Scotia,None,CA,None,False,None,"[{'id': 10, 'unit': 'ppm', 'count': 8653, 'ave...",None,"{'latitude': 49.9319, 'longitude': -96.8631}",2024-02-07T12:00:00+00:00,2016-05-06T21:00:00+00:00,12269,"[-96.8631, 49.9319, -96.8631, 49.9319]","[{'modelName': 'Government Monitor', 'manufact..."


In [41]:
# sub_missing_ids = locs_time3.id[~locs_time3.id.isin([*sensors.id.unique(), *sensors2.id.unique(), *sensor3.id.unique(), *sensor4.id.unique(), *sensor5.id.unique()])]
# sub_missing_ids

NameError: name 'locs_time3' is not defined

In [267]:
# print(len(locs_time.id.unique()))



# print("Length of missing IDS", len(ids))

# print("Lenght of IDS where no entries were found", len(no_entries_id))

# print("diff", (len(ids) - len(no_entries_id)))

964
Length of missing IDS 662
Lenght of IDS where no entries were found 33
diff 629


In [62]:
#locs_time[locs_time.id == 223021]

In [63]:
#get_measument_per_location(locations_id = 383253)

,none_found
0,383253
1,223021
2,399197
3,635630
4,1820911
5,635693
6,496097
7,382795
8,635657
9,1996960


In [228]:
# print("Length of missing IDS", len(ids))

# print("Lenght of IDS where no entries were found", len(no_entries_id))

# print("diff", (len(ids) - len(no_entries_id)))

Length of missing IDS 662
Lenght of IDS where no entries were found 87
diff 575


In [224]:
# ids = locs.id[~locs.id.isin([*sensors.id.unique(), *sensors2.id.unique(), *sensor3.id.unique(), *sensor4.id.unique() ])]

# print(len(locs.id.unique()))



# print("Length of missing IDS", len(ids))

# print("Lenght of IDS where no entries were found", len(no_entries_id))

# print("diff", (len(ids) - len(no_entries_id)))


# ids2 = locs.id[~locs.id.isin([*sensors.id.unique(), *sensors2.id.unique(), *sensor3.id.unique(), *sensor4.id.unique(), *no_entries_id])]

2232
Length of missing IDS 662
Lenght of IDS where no entries were found 512
diff 150


In [31]:
#1790156 #1481833
    
# locations_id = 1790156
    
# #get_measument_per_location("1481833")

# url = f"https://api.openaq.org/v2/averages?parameters_id=2&date_to=2023-09-30T00%3A00%3A00&date_from=2023-05-01T00%3A00%3A00&country=CA&locations_id={locations_id}&limit=1000&page=1"

# headers = {"accept": "application/json"}

# response = requests.get(url, headers=headers)

# test = response.json()
    

In [567]:
response.text

'{"message":"Too many requests"}'

In [ ]:
   
#     ## List of measurements
    
# url = "https://api.openaq.org/v2/measurements?date_from=2024-01-03T00%3A00%3A00Z&date_to=2024-01-10T15%3A53%3A00Z&limit=100&page=1&offset=0&sort=desc&radius=1000&order_by=datetime"

# headers = {"accept": "application/json"}

# response = requests.get(url, headers=headers)

# print(response.text)

In [75]:
# url = "https://api.openaq.org/v2/measurements?date_from=2023-05-01T00%3A00%3A00&date_to=2023-09-30T00%3A00%3A00&limit=7000&page=50&offset=0&sort=desc&has_geo=true&parameter_id=2&radius=1000&country=CA&order_by=datetime&entity=null"

# headers = {"accept": "application/json"}

# response = requests.get(url, headers=headers)

# test = response.json()
# measurements = pd.DataFrame(test['results'])

TypeError: string indices must be integers, not 'str'

In [437]:
# url = "https://api.openaq.org/v2/averages?parameters_id=2&date_to=2023-09-30T00%3A00%3A00&date_from=2023-05-01T00%3A00%3A00&country=CA&locations_id=65422&limit=1000&page=1"

# headers = {"accept": "application/json"}

# response = requests.get(url, headers=headers)

# test = response.json()
# output = pd.DataFrame(test['results'])

In [26]:
#output.id.unique()

In [27]:
#output.id.unique()

In [ ]:
#test["meta"]

In [28]:
#test

In [29]:
#sensors.hour.astype("datetime64[ns, UTC]").dt.hour.hist()

In [30]:
#sensors.groupby("id").average.median().hist()

In [233]:
#len(sub_missing_ids)

32

In [92]:
#### Actually subset the data. 

all_sens = pd.concat([sensors, sensors2, sensor3, sensor4, sensor5, sensor6], ignore_index = True)

In [93]:


all_sens = all_sens.drop_duplicates()

In [95]:
locs = unpack(locs, 'coordinates')
tmp = unpack(locs, 'manufacturers')
tmp = tmp.rename(columns={0:"manufacturers"})
locs = unpack(tmp, 'manufacturers')

In [98]:
#locs

In [99]:

locs = locs[["id", "city","lastUpdated", "firstUpdated", "measurements", "latitude", "longitude", "modelName", "manufacturerName"]]

In [448]:
locs

,id,city,lastUpdated,firstUpdated,measurements,latitude,longitude,modelName,manufacturerName
0,850723,None,2024-01-31T23:59:00+00:00,2023-04-12T23:00:27+00:00,1278420,50.673737,-120.322260,PurpleAir Sensor,PurpleAir
1,346487,None,2024-01-31T23:59:00+00:00,2022-06-02T14:20:05+00:00,1659960,43.838430,-66.110190,PurpleAir Sensor,PurpleAir
2,926806,None,2024-01-31T23:59:00+00:00,2022-08-24T20:44:49+00:00,1234698,56.256380,-120.830390,PurpleAir Sensor,PurpleAir
3,1465450,None,2024-01-31T23:59:00+00:00,2023-08-22T00:39:39+00:00,735059,50.993565,-118.204210,N/A,OpenAQ admin
4,337981,None,2024-01-31T23:58:59+00:00,2021-12-03T22:00:12+00:00,1684302,43.660812,-79.398380,PurpleAir Sensor,PurpleAir
...,...,...,...,...,...,...,...,...,...
2269,6138,None,2018-11-02T12:00:00+00:00,2018-08-02T13:00:00+00:00,2436,45.452318,-73.240116,Government Monitor,Unknown Governmental Organization
2270,1141358,None,2018-05-23T22:25:32+00:00,2018-05-23T22:25:32+00:00,12,43.529453,-80.287660,PurpleAir Sensor,PurpleAir
2271,1065,None,2018-01-31T14:00:00+00:00,2016-03-06T19:00:00+00:00,24405,49.887500,-124.558300,Government Monitor,Unknown Governmental Organization
2272,754,None,2017-03-30T16:00:00+00:00,2016-03-06T19:00:00+00:00,19385,45.957800,-66.647500,Government Monitor,Unknown Governmental Organization


In [100]:
all_sens = all_sens[['id', 'name', 'hour', 'day', 'month', 'year', 'hod',
       'dow', 'average', 'measurement_count', 'parameter', 'parameterId',
       'displayName', 'unit', 'first_datetime', 'last_datetime']]

all_sens = all_sens.drop_duplicates()

In [101]:
all_sens = all_sens[['id', 'name', 'hour','average', 'measurement_count', 'parameter',
       'displayName', 'unit']]

In [128]:
all_sens.city = all_sens.city.astype('str')

In [102]:
all_sens.id = all_sens.id.astype("float").apply(math.floor)

In [103]:
all_sens.id = all_sens.id.astype("str")

In [104]:
locs.id = locs.id.astype("str")

/tmp/ipykernel_161/278080568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locs.id = locs.id.astype("str")


In [105]:
all_sens = all_sens.merge(locs, on = "id", how = "left")

In [115]:
purple = all_sens[all_sens.modelName == 'PurpleAir Sensor']
purple.groupby(["id"]).average.mean()

id
1037243     5.723627
1037344    11.203105
1037393     9.809209
1037548    12.676657
1038146    10.346772
             ...    
930217     34.857183
938287     34.286842
997168     37.246896
997605     27.842277
998551     11.388155
Name: average, Length: 1236, dtype: float64

In [142]:
sens_mean = all_sens.groupby(["id",'name','parameter',
       'displayName', 'unit', 'city', 'lastUpdated', 'firstUpdated',
       'latitude', 'longitude', 'modelName',
       'manufacturerName' ]).average.mean()
sens_mean = sens_mean.reset_index()

In [143]:
# test_group = all_sens.groupby(["id",
#         'city', 'lastUpdated', 'firstUpdated',
#        'latitude', 'longitude', 'modelName',
#        'manufacturerName' ]).average.mean()
# test_group = test_group.reset_index()
# test_group.modelName.unique()

In [144]:
sens_mean.modelName.unique()

array(['Government Monitor', 'PurpleAir Sensor', 'N/A'], dtype=object)

In [145]:
max_sens = all_sens.groupby(["id"]).average.max()
max_sens = max_sens.reset_index()
max_sens = max_sens.rename(columns={"average":"max"})

sens_mean = sens_mean.merge(max_sens, on = ["id"])

In [146]:
mean_sens = all_sens.groupby(["id"]).average.mean()
mean_sens = mean_sens.reset_index()
mean_sens = mean_sens.rename(columns={"average":"mean"})

sens_mean = sens_mean.merge(mean_sens, on = ["id"])

In [147]:
quant_upper = all_sens.groupby(["id"]).average.quantile(0.975)
quant_upper = quant_upper.reset_index()
quant_upper = quant_upper.rename(columns={"average":"quant_97.5"})

sens_mean = sens_mean.merge(quant_upper, on = ["id"])

In [148]:
quant_lower = all_sens.groupby(["id"]).average.quantile(0.025)
quant_lower = quant_lower.reset_index()
quant_lower = quant_lower.rename(columns={"average":"quant_02.5"})

sens_mean = sens_mean.merge(quant_lower, on = ["id"])

In [149]:
quant_25 = all_sens.groupby(["id"]).average.quantile(0.25)
quant_25 = quant_25.reset_index()
quant_25 = quant_25.rename(columns={"average":"quant_25"})

sens_mean = sens_mean.merge(quant_25, on = ["id"])

In [150]:
quant_75 = all_sens.groupby(["id"]).average.quantile(0.75)
quant_75 = quant_75.reset_index()
quant_75 = quant_75.rename(columns={"average":"quant_75"})

sens_mean = sens_mean.merge(quant_75, on = ["id"])

In [151]:
min_sens = all_sens.groupby(["id"]).average.min()
min_sens = min_sens.reset_index()
min_sens = min_sens.rename(columns={"average":"min"})

sens_mean = sens_mean.merge(min_sens, on = ["id"])

In [152]:
sens_mean

,id,name,parameter,displayName,unit,city,lastUpdated,firstUpdated,latitude,longitude,modelName,manufacturerName,average,max,mean,quant_97.5,quant_02.5,quant_25,quant_75,min
0,1018,Parry Sound,pm25,PM2.5,µg/m³,ONTARIO,2023-07-17T16:00:00+00:00,2022-10-04T18:00:00+00:00,45.337800,-80.037800,Government Monitor,Unknown Governmental Organization,16.220299,250.0,16.220299,80.000,3.000000,5.000,15.0,1.000
1,1033,Peterborough,pm25,PM2.5,µg/m³,ONTARIO,2023-07-17T16:00:00+00:00,2016-03-06T19:00:00+00:00,44.300000,-78.350000,Government Monitor,Unknown Governmental Organization,15.409692,170.0,15.409692,90.875,2.000000,5.000,15.0,1.000
2,1037243,AQSU-7260,pm25,PM2.5,µg/m³,None,2024-02-03T03:22:33+00:00,2023-06-03T11:44:51+00:00,44.800720,-63.659683,PurpleAir Sensor,PurpleAir,5.723627,40.0,5.723627,29.000,0.000000,0.033,7.3,0.000
3,1037344,AQSU-93FA,pm25,PM2.5,µg/m³,None,2023-07-08T14:53:52+00:00,2023-05-30T21:55:38+00:00,44.608300,-63.478900,PurpleAir Sensor,PurpleAir,11.203105,130.0,11.203105,43.000,0.351250,3.525,15.0,0.075
4,1037393,AQSU-9157,pm25,PM2.5,µg/m³,None,2024-02-07T14:10:05+00:00,2023-05-29T21:26:05+00:00,44.712800,-63.678600,PurpleAir Sensor,PurpleAir,9.809209,160.0,9.809209,35.550,0.096450,3.500,13.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603,958,Saskatoon,pm25,PM2.5,µg/m³,SASKATCHEWAN,2024-02-07T12:00:00+00:00,2016-03-06T19:00:00+00:00,52.136055,-106.662796,Government Monitor,Unknown Governmental Organization,26.270567,560.0,26.270567,150.000,3.000000,8.000,25.0,2.000
1604,988,Langdale Elementary,pm25,PM2.5,µg/m³,BRITISH COLUMBIA,2024-02-07T12:00:00+00:00,2016-03-06T19:00:00+00:00,49.438848,-123.479185,Government Monitor,Unknown Governmental Organization,7.557012,110.0,7.557012,33.000,2.300000,4.200,7.5,0.900
1605,997168,Aspen Trails,pm25,PM2.5,µg/m³,None,2024-02-07T14:11:42+00:00,2023-05-24T18:38:27+00:00,53.560680,-113.283710,PurpleAir Sensor,PurpleAir,37.246896,490.0,37.246896,210.000,1.200000,8.700,43.0,0.000
1606,997605,3023 Birkenhead dr,pm25,PM2.5,µg/m³,None,2024-02-07T14:09:58+00:00,2023-05-24T18:35:43+00:00,50.656776,-120.226140,PurpleAir Sensor,PurpleAir,27.842277,840.0,27.842277,150.750,0.064625,4.000,28.0,0.000


array(['Government Monitor'], dtype=object)